In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision.datasets import ImageFolder
from torchvision import transforms
from tqdm import tqdm


In [48]:
data_path = 'E:\cifar-10'

train_label = pd.read_csv("E:\cifar-10\\trainLabels.csv")


In [29]:

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(32),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.3,hue=0.2), 
    
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465],
                        [0.2023, 0.1994, 0.2010])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465],
                        [0.2023, 0.1994, 0.2010])
])

In [50]:
from random import random
import os

def copy_file(source_directory, destination_directory, filename):
    """
    Utility function used to copy a file from a source_directory to a destination_directory
    """
    destination_directory.mkdir(parents=True, exist_ok=True)
    shutil.copy(source_directory/filename, destination_directory/filename)
    
def organize_train_valid_dataset(root, labels, valid_probability=0.1):
    """
    Creates the train, train_valid and valid folders respecting PyTorch's ImageDataset structure, performing
    train/validation split based on the given percentage
    """
    source_directory = root
    
    with os.scandir(source_directory) as it:
        for entry in it:
            if entry.is_file():
                img_index = entry.name.split('.')[0]  # The index is the name of the image except the extension
                img_class = labels[labels.id==int(img_index)].label.values[0]  # Find the class by looking up the index in the DF
                
                # Randomly assign the image to the valid dataset with probability 'valid_probability'
                channel = Path('train') if random()>valid_probability else Path('valid')
                destination_directory = root/channel/img_class
                
                # Copy the image to either the train or valid folder, and also to the train_valid folder
                copy_file(source_directory, destination_directory, entry.name)
                copy_file(source_directory, root/'train_valid'/img_class, entry.name)

def organize_test_dataset(root):
    """
    Creates the test folder respecting PyTorch's ImageDataset structure, using a dummy 'undefined' label
    """
    source_directory = root/'original_test'
        
    with os.scandir(source_directory) as it:
        for entry in it:
            if entry.is_file():
                img_index = entry.name.split('.')[0]  # The index is the name of the image except the extension

                channel = Path('test')
                destination_directory = root/channel/'undefined'

                copy_file(source_directory, destination_directory, entry.name)

In [52]:
from pathlib import Path
import shutil
root = Path("E:\cifar-10\\train")
valid_probability = 0.1
organize_train_valid_dataset(root, train_label, valid_probability)

# Create the test folder structure
#organize_test_dataset(r)

In [49]:
train_label

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile
